In [300]:
"""
Скрипт для получения списка учеников посетивших 2 и более профпроб, и списка участников по муниципалитетам школам,
количества участников по муниципалитетам школам
"""

'\nСкрипт для получения списка учеников посетивших 2 и более профпроб, и списка участников по муниципалитетам школам,\nколичества участников по муниципалитетам школам\n'

In [301]:
import pandas as pd
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.utils import get_column_letter
import time
import re
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
pd.options.mode.chained_assignment = None

In [302]:
path_to_end_folder = 'data'

In [303]:
#df = pd.read_excel('data/students.xlsx',skiprows=3)
df = pd.read_excel('data/students_all.xlsx',skiprows=3)
t = time.localtime()
current_time = time.strftime('%H_%M_%S',t)
wb = openpyxl.Workbook() # файл для записавшихся на 2 и более профпробы


In [304]:

wb.create_sheet('Записавшиеся на 2 и более проб',index=0)


<Worksheet "Записавшиеся на 2 и более проб">

In [305]:
df.shape

(17509, 45)

In [306]:
# Заполняем пустые ячейки
df['названия'] = df['названия'].fillna('Пусто')
df['даты прохождения'] = df['даты прохождения'].fillna('Пусто')
df['темы диагностик'] = df['темы диагностик'].fillna('Пусто')
df['даты пройденных диагностик'] = df['даты пройденных диагностик'].fillna('Пусто')


In [307]:
# создаем копию датафрейма для подсчета диагностик
diag_df = df.copy()

In [308]:
# разворачиваем датафрейм 
flat_rows = [] # список для хранения строк 
for index,row in df.iterrows():
    name_probs = row['названия'].split(',') # названия проб
    date_probs = row['даты прохождения'].split(',') # даты прохождения
    for idx,name in enumerate(name_probs):
        new_row = row.copy() # копируем строку
        new_row['названия'] = name
        try:
            new_row['даты прохождения'] = date_probs[idx]
            flat_rows.append(new_row)
        except IndexError:
            new_row['даты прохождения'] = date_probs[idx-1]
            flat_rows.append(new_row)
# Создание нового датафрейма на основе новых строк
flat_df = pd.DataFrame(flat_rows)

In [309]:
flat_df['даты прохождения'] = flat_df['даты прохождения'].astype(str)
flat_df['даты прохождения'] = flat_df['даты прохождения'].apply(lambda x:x.strip())
flat_df =flat_df[flat_df['даты прохождения'] != 'Пусто'] # удаляем тех кто не проходил 
flat_df['даты прохождения'] = pd.to_datetime(flat_df['даты прохождения'],dayfirst=True,errors='ignore') #приводим к типу дата

In [310]:
flat_df = flat_df[(flat_df['даты прохождения'] > '2023-09-01') & (flat_df['даты прохождения']< '2023-12-31')] # отбираем только те профпробы которые были этой осенью


In [311]:
flat_df.shape

(511, 45)

In [312]:
dupl_df=flat_df[flat_df.duplicated(subset=['ФИО','Дата рождения'],keep=False)] # отбираем дубликаты
lst_out = ['Муниципалитет','ФИО','Дата рождения','образовательная организация','класс (без буквы)','буква класса',
           'названия','даты прохождения','ответственный педагог-навигатор']

dupl_df = dupl_df[lst_out]

dupl_df = dupl_df.sort_values(by=['ФИО','даты прохождения'],ascending=True)
dupl_df['даты прохождения'] = dupl_df['даты прохождения'].dt.strftime('%d.%m.%Y')

for row in dataframe_to_rows(dupl_df,index=False,header=True):
    wb['Записавшиеся на 2 и более проб'].append(row)
for column in wb['Записавшиеся на 2 и более проб'].columns:
    max_length = 0
    column_name = get_column_letter(column[0].column)
    for cell in column:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = (max_length + 2)
    wb['Записавшиеся на 2 и более проб'].column_dimensions[column_name].width = adjusted_width
wb.save(f'{path_to_end_folder}\Записавшиеся на 2 и более профпроб {current_time}.xlsx')



In [313]:
# разворачиваем датафрейм 
diag_rows = [] # список для хранения строк 
for index,row in diag_df.iterrows():
    name_diag = row['темы диагностик'].split(',') # названия проб
    date_diag = row['даты пройденных диагностик'].split(',') # даты прохождения
    for idx,name in enumerate(name_diag):
        new_row = row.copy() # копируем строку
        new_row['темы диагностик'] = name
        try:            
            new_row['даты пройденных диагностик'] = date_diag[idx]
            diag_rows.append(new_row)
        except IndexError:
            new_row['даты пройденных диагностик'] = date_diag[-1]
            diag_rows.append(new_row)
# Создание нового датафрейма на основе новых строк
diag_df = pd.DataFrame(diag_rows)



In [314]:
diag_df.shape

(29677, 45)

In [315]:
diag_df['даты пройденных диагностик'] = diag_df['даты пройденных диагностик'].astype(str)
diag_df['даты пройденных диагностик'] = diag_df['даты пройденных диагностик'].apply(lambda x:x.strip())
diag_df =diag_df[diag_df['даты пройденных диагностик'] != 'Пусто'] # удаляем тех кто не проходил 
diag_df['даты пройденных диагностик'] = pd.to_datetime(diag_df['даты пройденных диагностик'],dayfirst=True,errors='ignore') #приводим к типу дата

diag_df = diag_df[(diag_df['даты пройденных диагностик'] > '2023-09-01') & (diag_df['даты пройденных диагностик']< '2023-12-01')] # отбираем только те профпробы которые были этой осенью


In [316]:
diag_df.shape

(8280, 45)

In [317]:
diag_df.columns

Index(['№пп', 'Федеральный округ', 'Регион', 'Муниципалитет', 'ФИО',
       'Дата рождения', 'образовательная организация',
       'Дата (год) регистрации на платформе', 'В архиве с какого года',
       'Статус авторизации в системе', 'Дата последней активности',
       'Статус верификации', 'класс (без буквы)', 'буква класса', 'группы',
       'дата рождения/ возраст', 'пол', 'почта, телефон', 'наличие ОВЗ',
       'нозология', 'скан согласия родителей на обработку перс.данных',
       'количество пройденных диагностик', 'темы диагностик',
       'даты пройденных диагностик', 'дата получения рекомендации',
       'рекомендуемые среды', 'кол-во посещенных профпроб в текущем году',
       'названия', 'даты прохождения',
       'кол-во посещенных вирт. профпроб в текущем году', 'названия.1',
       'даты прохождения.1',
       'кол-во посещенных партнерск. профпроб в текущем году', 'названия.2',
       'даты прохождения.2', 'кол-во посещенных уроков в текущем году',
       'названия урок

In [318]:
drop_dupl_diag_df = diag_df.drop_duplicates(subset=['ФИО','Дата рождения','образовательная организация'])

In [319]:
drop_dupl_diag_df.shape

(3098, 45)

In [320]:
#делаем сводную таблицу
svod_diag_df = pd.pivot_table(drop_dupl_diag_df,
                             index=['Муниципалитет','образовательная организация'],
                             values='ФИО',
                             aggfunc='count',
                             margins=True)

In [321]:
svod_diag_df= svod_diag_df.reset_index() # вытаскиваеем индекс
svod_diag_df.rename(columns={'ФИО':'Количество прошедших хотя бы 1 диагностику'},inplace=True)
svod_diag_df.sort_values(by=['Муниципалитет','Количество прошедших хотя бы 1 диагностику',],
                        ascending=False,inplace=True)

In [322]:
svod_diag_df.head()

,Муниципалитет,образовательная организация,Количество прошедших хотя бы 1 диагностику
109,город Улан-Удэ,МАОУ «СОШ №54 г. Улан-Удэ»,115
89,город Улан-Удэ,"МАОУ ""СОШ № 49 г. Улан-Удэ""",83
107,город Улан-Удэ,МАОУ «СОШ №4 г. Улан-Удэ»,79
110,город Улан-Удэ,МАОУ «СОШ №65 г. Улан-Удэ им.Г.С. Асеева»,65
85,город Улан-Удэ,"МАОУ ""СОШ № 25""",52


In [323]:
lst_sheet = svod_diag_df['Муниципалитет'].unique() # список значений


In [324]:
wb_diag = openpyxl.Workbook() # создаем файл
wb_diag.create_sheet(title='Общий свод',index=0)
for row in dataframe_to_rows(svod_diag_df,index=False,header=True):
    wb_diag['Общий свод'].append(row)

# Устанавливаем автоширину для каждой колонки
for column in wb_diag['Общий свод'].columns:
    max_length = 0
    column_name = get_column_letter(column[0].column)
    for cell in column:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = (max_length + 2)
    wb_diag['Общий свод'].column_dimensions[column_name].width = adjusted_width


used_name_sheet = set() # множество для хранения значений которые уже были использованы
name_column = 'Муниципалитет'
for idx,value in enumerate(lst_sheet,1):
    temp_df = svod_diag_df[svod_diag_df[name_column] == value] # отфильтровываем по значению
    short_value = value[:20] # получаем обрезанное значение
    short_value = re.sub(r'[\[\]\'+()<> :"?*|\\/]', '_', short_value)

    if short_value in used_name_sheet:
        short_value = f'{short_value}_{idx}' # добавляем окончание
    wb_diag.create_sheet(short_value,index=idx) # создаем лист
    used_name_sheet.add(short_value)
    # создаем строку Сумма
    temp_df.loc['Сумма'] = temp_df['Количество прошедших хотя бы 1 диагностику'].sum()
    temp_df.loc[temp_df.index[-1],'Муниципалитет'] = None
    temp_df.loc[temp_df.index[-1],'образовательная организация'] = None
    for row in dataframe_to_rows(temp_df,index=False,header=True):
        wb_diag[short_value].append(row)

    # Устанавливаем автоширину для каждой колонки
    for column in wb_diag[short_value].columns:
        max_length = 0
        column_name = get_column_letter(column[0].column)
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2)
        wb_diag[short_value].column_dimensions[column_name].width = adjusted_width

wb_diag.create_sheet()
        
        
if 'All' and 'Sheet' in wb_diag.sheetnames: # удаляем лишние листы    
    del wb_diag['All']
    del wb_diag['Sheet']

wb_diag.save(f'{path_to_end_folder}\Свод по диагностике {current_time}.xlsx')



In [325]:

print('Lindy Booth')

Lindy Booth
